In [ ]:
## combining Clinvar info with UNEECON info

In [1]:
import pandas as pd
import numpy as np
import gzip
import io
import os
import random

In [2]:
column_list = ['chrom','pos','ID','ref','alt','qual','filter','info']
clinvar = pd.read_csv('/storage/home/jkl5991/work/project/original_data/clinvar_20200113.vcf', sep='\t', header=None, comment='#', names = column_list)
clinvar.head()
clinvar.columns = column_list
clinvar = clinvar[clinvar['ref'].str.len() == clinvar['alt'].str.len()]

/storage/home/jkl5991/.conda/envs/sanEnv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
# building dictionary for pathogenic or benign
# building dictionary for dominant/recessive
ref_path={}
ref_do ={}
c = 0
controdict = 0
# to irerate over rows in Pandas DataFrame
for index,row in clinvar.iterrows():
    a = "-".join(["chr" + str(row['chrom']), str(row['pos']-1), str(row['pos']), row['ref'], row['alt']])
    c += 1
    if 'CLNSIG=Conflicting_interpretations_of_pathogenicity' not in row['info']:
        if "benign" in row['info'] or "Benign" in row['info']:
            ref_path[a] = 0
        if "pathogenic" in row['info'] or "Pathogenic" in row['info']:
            ref_path[a] = 1
        if 'dominant' in row['info']:
            ref_do[a] = 1
        elif "recessive" in row['info']:
            ref_do[a] = 0
        if (('dominant' in row['info']) and ('recessive' in row['info'])):
            controdict +=1
    if c % 50000 == 0:
        print('checked {} items'.format(c))
print('finished. Checked {} items'.format(c))
print('controdicted inheritance mode = ', controdict)

checked 50000 items
checked 100000 items
checked 150000 items
checked 200000 items
checked 250000 items
checked 300000 items
checked 350000 items
checked 400000 items
checked 450000 items
checked 500000 items
checked 550000 items
finished. Checked 551918 items
controdicted inheritance mode =  947


In [4]:
# creating dominance.tsv, benign.tsv, unannotated.tsv
with open('dominance.tsv', 'w') as dominance, open('benign.tsv','w') as benign, open('unannotated.tsv','w') as unannotated:
    predict = gzip.open('/storage/home/jkl5991/work/project/original_data/prediction_input.tsv.gz','r')
    column_name =['location', 'accession_num', 'clinvar_result', 'SIFT_pred', 'LRT_pred', 'MA_pred', 'PROVEN_pred',
               'SLR_score', 'SIFT_score', 'LRT_omega', 'MA_score', 'PROVEN_score', 'Grantham', 'HMMEntropy',
               'HMMRelEntropy', 'PredRSAB', 'PredRSAI', 'PredRSAE', 'PredBFactorF', 'PredBFactorM', 'PredBFactorS',
               'PredStabilityH', 'PredStabilityM', 'PredStabilityL', 'PredSSE', 'PredSSH', 'PredSSC', 'dscore',
               'phyloP_pri', 'phyloP_mam', 'phyloP_ver', 'RNA_seq','dominantORrecessive\n'] 
    dominance.write('\t'.join(column_name))
    benign.write('\t'.join(column_name))
    unannotated.write('\t'.join(column_name))
    print("start running...")
    count_dominance = count_benign = count_unannotated = 0
    for predictline  in predict:
        predictline = predictline.decode('UTF-8')
        predict_split = predictline.split('\t')
        if predict_split[0] in ref_do and predict_split[0] in ref_path:
            outputline = "\t".join(predict_split[:2]+[str(ref_path[predict_split[0]])]+predict_split[2:30]+[predict_split[30].strip()] + [str(ref_do[predict_split[0]])])+'\n'
            dominance.write(outputline)
            count_dominance += 1

        if  predict_split[0] in ref_path and predict_split[0] not in ref_do:
            outputline = "\t".join(predict_split[:2]+[str(ref_path[predict_split[0]])]+predict_split[2:30]+[predict_split[30].strip()] + ["NA"])+"\n"
            unannotated.write(outputline)
            count_unannotated += 1  
            if ref_path[predict_split[0]] == 0:  
                benign.write(outputline)
                count_benign += 1  
        
                
    print("finished")
    print("wrote {} lines in dominance.tsv".format(count_dominance))
    print("wrote {} lines in benign.tsv".format(count_benign))
    print("wrote {} lines in unannotated.tsv".format(count_unannotated))
    predict.close() 

start running...
finished
wrote 3001 lines in dominance.tsv
wrote 48076 lines in benign.tsv
wrote 79393 lines in unannotated.tsv


In [6]:
# balanced file
dominance = pd.read_csv("dominance.tsv", sep = "\t")
benign = pd.read_csv("benign.tsv",sep = "\t")
print(dominance.shape, benign.shape)

random.seed(42)
benign_random = benign.sample(n = 1713) #2357-644
print(benign_random.shape)

dominance_balanced = dominance.append(benign_random, sort = False)
dominance_balanced[dominance_balanced['dominantORrecessive'].isna()].shape # there are 1703 added data from benign.tsv
print(dominance_balanced.shape)
dominance_balanced.to_csv(r'dominance_balanced.tsv', sep = "\t", index = False)

(3001, 33) (48076, 33)
(1713, 33)
(4714, 33)


In [7]:
dominant = dominance[dominance['dominantORrecessive'] ==1]
recessive = dominance[dominance['dominantORrecessive'] ==0]
dominant[dominant['clinvar_result'] == 0].shape
recessive[recessive['clinvar_result'] == 1].shape
print(dominance[dominance['clinvar_result'] ==1 ].shape)
print(dominance[dominance['clinvar_result'] ==0 ].shape)
print(dominance[dominance['dominantORrecessive'] == 1].shape)
print(dominance[dominance['dominantORrecessive'] == 0].shape)
# print(dominance.shape)




(2357, 33)
(644, 33)
(1273, 33)
(1728, 33)


In [8]:
#omit file
unannotated = pd.read_csv('unannotated.tsv', sep = "\t")
unannotated_omit = unannotated.loc[~unannotated['location'].isin(benign_random['location'])]
unannotated_omit = unannotated_omit.drop(["dominantORrecessive"],axis = 1)
print(unannotated_omit.shape)  #79393 - 1713 = 77680
unannotated_omit.to_csv(r'unannotated_omit.tsv', sep = '\t', index = False)

(77680, 32)


In [9]:
np.sum(unannotated_omit['clinvar_result'])

31317

In [ ]:
# tabix uneecon
!cat dominance_balanced.tsv |cut -f 1| grep -|tr -d 'chr'|cut -d '-' -f 1,2,3|tr '-' '\t' > location_balanced.bed
!sortBed -i location_balanced.bed |mergeBed > location_balanced_sorted.bed
!tabix ~/work/project/original_data/UNEECON.tsv.gz -R location_balanced_sorted.bed > tabix_balanced.tsv

!cat unannotated_omit.tsv |cut -f 1| grep -|tr -d 'chr'|cut -d '-' -f 1,2,3|tr '-' '\t' > location_omit.bed    
!sortBed -i location_omit.bed |mergeBed > location_omit_sorted.bed 
!tabix ~/work/project/original_data/UNEECON.tsv.gz -R location_omit_sorted.bed > tabix_omit.tsv 

In [98]:
#balanced file + uneecon
uneecon = pd.read_csv("tabix_balanced.tsv", sep = '\t', header = None, names = ['chrom','loc','ref','alt','score'])
uneecon['loc-1'] = uneecon['loc']-1
uneecon['match'] ='chr' + uneecon['chrom'].map(str)+ '-' + uneecon['loc-1'].map(str)+ '-' + uneecon['loc'].map(str)+ '-' + uneecon['ref'].map(str)+ '-' + uneecon['alt'].map(str)


df = pd.DataFrame({'location' : uneecon['match'], 'UNEECON':uneecon['score']})
balanced_uneecon = pd.merge(dominance_balanced,df, on=['location'])

balanced_uneecon.to_csv('dominance_balanced_uneecon.tsv', sep = "\t", index = False)


In [103]:
# unannotated file + UNEECON
unannotated_omit= pd.read_csv('unannotated_omit.tsv', sep = '\t')
uneecon = pd.read_csv("tabix_omit.tsv", sep = '\t', header = None, names = ['chrom','loc','ref','alt','score'])
uneecon['loc-1'] = uneecon['loc']-1
uneecon['match'] ='chr' + uneecon['chrom'].map(str)+ '-' + uneecon['loc-1'].map(str)+ '-' + uneecon['loc'].map(str)+ '-' + uneecon['ref'].map(str)+ '-' + uneecon['alt'].map(str)
df = {'location' : uneecon['match'], 'UNEECON':uneecon['score']}
df = pd.DataFrame(df)
omit_uneecon = pd.merge(unannotated_omit,df, on=['location'])
omit_uneecon.to_csv(r'unannotated_omit_uneecon.tsv', sep = '\t', index = False)

In [112]:
# create dominant and recessive seperate file #with UNEECON score
#balanced_uneecon = pd.read_csv('dominance_balanced_uneecon.tsv', sep = '\t')
benign = balanced_uneecon.loc[balanced_uneecon['dominantORrecessive'].isna()]

random.seed(42)
randombenign4domi = benign.sample(n = 677) #975 - 298 = 677
print(randombenign4domi.shape)

randombenign4recess = benign.loc[~benign['location'].isin(randombenign4domi['location'])]
print(randombenign4recess.shape)

dominant = balanced[balanced['dominantORrecessive']==1]
dominant = pd.concat([dominant,randombenign4domi])
dominant = dominant.drop(["dominantORrecessive"],axis = 1)
print(dominant.shape)

recessive = balanced[balanced['dominantORrecessive']==0]
recessive = pd.concat([recessive,randombenign4recess])
recessive = recessive.drop(["dominantORrecessive"],axis = 1)
print(recessive.shape)

#dominant.to_csv(r'dominant_uneecon.tsv', sep = "\t", index = False)
#recessive.to_csv(r'recessive_uneecon.tsv', sep = "\t", index = False)

(677, 34)
(1036, 34)
(1950, 33)
(2764, 33)


In [113]:
#standardize 
mean = omit_uneecon.mean(axis = 0, numeric_only = float)
std = omit_uneecon.std(axis = 0, numeric_only = float)
drop_col = ['clinvar_result','SIFT_pred','LRT_pred','MA_pred','PROVEN_pred']
mean = mean.drop(drop_col)
std = std.drop(drop_col)

omit_std = (omit_uneecon.iloc[:,7:] - mean)/std
dominant_std = (dominant.iloc[:,7:] - mean)/std
recessive_std = (recessive.iloc[:,7:] - mean)/std

omit_std = pd.concat([omit_uneecon.iloc[:,:7], omit_std], axis = 1)
dominant_std = pd.concat([dominant.iloc[:,:7], dominant_std], axis = 1)
recessive_std = pd.concat([recessive.iloc[:,:7], recessive_std], axis = 1)

In [116]:
print(omit_std.shape)
print(dominant_std.shape)
print(recessive_std.shape)

(77680, 33)
(1950, 33)
(2764, 33)


In [117]:
omit_std.to_csv('unannotated_omit_std.tsv', sep = '\t',index = False)
dominant_std.to_csv('dominant_std.tsv', sep = '\t',index = False)
recessive_std.to_csv("recessive_std.tsv",sep = '\t',index = False)

In [2]:
omit_std = pd.read_csv('unannotated_omit_std.tsv', sep = '\t')
dominant_std = pd.read_csv('dominant_std.tsv', sep = '\t')
recessive_std = pd.read_csv("recessive_std.tsv", sep = '\t')

In [3]:
from test_function import *
omit_std = preprocess(omit_std, 'clinvar_result',Balanced = True)

Unbalanced,
with num of 0 is 46363, num of 1 is 31317.

start balancing...
with num of 0 is 31317, num of 1 is 31317.



In [4]:
omit_std = omit_std.rename(columns = {'PROVEN_score':'PROVEAN_score', 'PROVEN_pred':'PROVEAN_pred'})
dominant_std = dominant_std.rename(columns = {'PROVEN_score':'PROVEAN_score', 'PROVEN_pred':'PROVEAN_pred'})
recessive_std = recessive_std.rename(columns = {'PROVEN_score':'PROVEAN_score', 'PROVEN_pred':'PROVEAN_pred'})

In [6]:
dominant_std.columns

Index(['location', 'accession_num', 'clinvar_result', 'SIFT_pred', 'LRT_pred',
       'MA_pred', 'PROVEAN_pred', 'SLR_score', 'SIFT_score', 'LRT_omega',
       'MA_score', 'PROVEAN_score', 'Grantham', 'HMMEntropy', 'HMMRelEntropy',
       'PredRSAB', 'PredRSAI', 'PredRSAE', 'PredBFactorF', 'PredBFactorM',
       'PredBFactorS', 'PredStabilityH', 'PredStabilityM', 'PredStabilityL',
       'PredSSE', 'PredSSH', 'PredSSC', 'dscore', 'phyloP_pri', 'phyloP_mam',
       'phyloP_ver', 'RNA_seq', 'UNEECON'],
      dtype='object')

In [7]:
recessive_std.columns

Index(['location', 'accession_num', 'clinvar_result', 'SIFT_pred', 'LRT_pred',
       'MA_pred', 'PROVEAN_pred', 'SLR_score', 'SIFT_score', 'LRT_omega',
       'MA_score', 'PROVEAN_score', 'Grantham', 'HMMEntropy', 'HMMRelEntropy',
       'PredRSAB', 'PredRSAI', 'PredRSAE', 'PredBFactorF', 'PredBFactorM',
       'PredBFactorS', 'PredStabilityH', 'PredStabilityM', 'PredStabilityL',
       'PredSSE', 'PredSSH', 'PredSSC', 'dscore', 'phyloP_pri', 'phyloP_mam',
       'phyloP_ver', 'RNA_seq', 'UNEECON'],
      dtype='object')

In [8]:
omit_std.to_csv('unannotated_omit_std.tsv', sep = '\t',index = False)
dominant_std.to_csv('dominant_std.tsv', sep = '\t',index = False)
recessive_std.to_csv("recessive_std.tsv",sep = '\t',index = False)